# 컴퓨터 파워 Up! 파이썬으로 멀티프로세싱/멀티스레딩 하기

**Context**

1. [멀티태스킹 (1) 멀티태스킹이란?](#멀티태스킹-1-멀티태스킹이란)
2. [멀티태스킹 (2) 프로세스, 쓰레드, 프로파일링](#멀티태스킹-2-프로세스-쓰레드-프로파일링)
3. [멀티태스킹 (3) Scale Up vs Scale Out](#멀티태스킹-3-scale-up-vs-scale-out)
4. [파이썬에서 멀티스레드 사용하기 (1) 스레드 생성](#파이썬에서-멀티스레드-사용하기-1-스레드-생성)
5. [파이썬에서 멀티스레드 사용하기 (2) 스레드 생성 및 사용](#파이썬에서-멀티스레드-사용하기-2-스레드-생성-및-사용)
6. [파이썬에서 멀티프로세스 사용하기](#파이썬에서-멀티프로세스-사용하기)
7. [파이썬에서 스레드/프로세스 풀 사용하기](#파이썬에서-스레드프로세스-풀-사용하기)
8. [실전 예제 (1) futures 모듈](#실전-예제-1-futures-모듈)
9. [실전 예제 (2) 튜토리얼](#실전-예제-2-튜토리얼)


## 멀티태스킹 (1) 멀티태스킹이란?

우리는 TV를 보면서 밥을 먹는 일을 하기도 하고, 설거지를 하면서 라디오를 듣기도 합니다. 우리는 이렇게 2가지 작업을 동시에 할 수 있습니다. 컴퓨터에게도 이렇게 일을 시키고 싶어지지 않나요? 사실 어느 정도 가능합니다. 특히 컴퓨터에서는 이렇게 여러 가지 작업을 동시에 하는 것을 "멀티태스킹"이라고 합니다.

### 동시성, 병렬성

멀티태스킹에는 2가지 동작 방식이 있는데요, 바로 동시성과 병렬성입니다.

- **동시성(Concurrency)**: 배달과 그릇 찾기라는 2가지 작업을 한다고 할 때, 한 명의 직원을 고용해도 사람이 바삐 움직여 주기만 하면 배달, 그릇찾기, 배달, 그릇 찾기 등 여러 가지 일을 동시에 수행할 수 있습니다. 물론 진짜로 동시에 하는 건 아니지만, 오늘 하루의 일과를 보고 받을 때에는 2가지 일이 동시에 종료가 되었다고 말하겠지요. 이렇게 정말로 동시에 하는 건 아니지만 여러 가지 일을 빠르게 번갈아가며 수행해 동시에 수행하는 것처럼 일하는 것을 동시성이라고 합니다.
- **병렬성(Parallelism)**: 업무를 분담해서 할 수도 있을 겁니다. 두 명의 직원을 고용해서 한 명은 그릇 찾기 일만 하고 한 명은 배달 일만 하는 거예요. 이런 경우에는 정말 동시에 2가지 일이 진행될 수 도 있겠네요. 이 경우 일이 병렬적으로 처리된다고 말합니다.

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/_2020-05-26__6.22.51.max-800x600.png)




### 동기 vs 비동기 (Synchronous vs Asynchronous)

동시성에서 주로 다루게 될 개념입니다. 배달과 그릇 찾기 2가지 일을 반복적으로 한다고 해볼게요. 배달하고 그릇을 찾으러 갔는데 아직 그릇이 안 나왔네요. 아마 아직도 먹고 있나봐요. 그럼 기다렸다가 그릇을 찾는 게 좋을까요, 아니면 다른 곳을 먼저 배달하고 그릇을 찾는 게 더 좋을까요?

이렇게 어떤 일을 바로 하지 못하고 대기해야 하는 일을 일컬을 때 컴퓨터에서는 *"바운드(bound)되었다"* 라는 표현을 많이 씁니다. 이렇게 바운드되고 있으면 이걸 계속 기다려야 할지 아니면 종료되는 사이에 다른 걸 실행하는것이 좋을지 고민이 되지요.

개념을 일반화 시켜 이야기하면 작동하는 방식에 따라 앞 작업이 종료되기를 무조건 기다렸다가 다음 작업을 수행하는 것은 동기(synchronized) 방식이라고 이야기합니다. 기다리는 동안 다른 일을 처리하는 것을 비동기(asynchronous) 방식이라고 이야기합니다.

- **동기(Synchronous)** : 어떤 일이 순차적으로 실행됨, 요청과 요청에 대한 응답이 동시에 실행됨 (따라서 요청에 지연이 발생하더라도 계속 대기한다.)
- **비동기(Asynchronous)** : 어떤 일이 비순차적으로 실행됨, 요청과 요청에 대한 응답이 동시에 실행되지 않음. 특정 코드의 연산이 끝날 때까지 코드의 실행을 멈추지 않고 다음 코드를 먼저 실행하며, 중간에 실행되는 코드는 주로 콜백함수로 연결하기도 한다.

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/_2020-05-26__6.22.57.max-800x600.png)


### I/O Bound vs CPU Bound

컴퓨터가 일을 수행하면서 뭔가 기다릴 때, 즉 속도에 제한이 걸릴 때는 2가지 경우에 해당하는 경우가 대부분입니다.

- **I/O 바운드** : 입력과 출력에서의 데이터(파일)처리에 시간이 소요될 때.
- **CPU 바운드** : 복잡한 수식 계산이나 그래픽 작업과 같은 엄청난 계산이 필요할 때.

아래 링크에서 자세한 내용을 알아봅시다.

- [What do the terms "CPU Bound" and "I/O Bound" mean?](https://stackoverflow.com/questions/868568/what-do-the-terms-cpu-bound-and-i-o-bound-mean)

## 멀티태스킹 (2) 프로세스, 쓰레드, 프로파일링

우리는 PPT 문서 작업을 하면서, 노래를 듣기도 하고, 웹 브라우저를 이용해 검색을 하기도 합니다. 요즘 컴퓨터에서는 이렇게 여러 개 프로그램을 동시에 실행시키는 멀티태스킹이 기본 사양입니다.

맥(Mac)의 `활성 상태 보기`(Activity Monitor)나 윈도우(Windows)의 `작업 관리자`(Task Manager)를 통해 현재 실행되고 있는 프로그램의 상태를 확인할 수 있습니다.

눈으로 확인해 봤는데요, 우리가 프로그램을 실행할 때 컴퓨터에서는 어떤 일이 일어날까요?


### Process (프로세스)

> An Instance of a program 

하나의 프로그램을 실행할 때, 운영체제는 한 프로세스를 생성합니다. 프로세스는 운영체제의 커널(Kernel)에서 시스템 자원(CPU, 메모리, 디스크) 및 자료구조를 이용합니다.

프로세스는 "<u>프로그램을 구동하여 프로그램 자체와 프로그램의 상태가 메모리 상에서 실행되는 작업 단위</u>"를 지칭합니다. 예를 들어, 하나의 프로그램을 한 번 구동하면 하나의 프로세스가 메모리 상에서 실행되지만 여러 번 구동하면 여러 개의 프로세스가 실행됩니다.

파이썬에서는 os 라는 모듈에서 프로세스 관련 정보를 얻을 수 있습니다.

In [1]:
import os

# process ID
print(os.getpid())

# user ID
print(os.getuid())

# group ID
print(os.getgid())

# 현재 작업중인 디렉토리
print(os.getcwd())

19929
1000
1000
/home/aiffel0039/Documents/GitHub/aiffel/Week6


방금 `os.getpid()`를 통해 얻어진 pid는 어떤 프로세스의 것일까요? 터미널을 열어 다음과 같이 해당 프로세스의 상세 정보를 확인해 봅시다. 저의 경우 pid 값이 19929가 얻어졌습니다.

```bash
$ ps -ef | grep 19929
```

네! 바로 지금 돌아가고 있는 노트북이 프로세스 19929였습니다.


### Thread(스레드)

프로그램 내에서도 많은 작업들이 있습니다. 스레드(thread)는 어떠한 *프로그램 내*에서, 특히 프로세스 내에서 실행되는 흐름의 단위입니다. 예를 들어 요리를 만드는 프로그램이라고 한다면 김밥, 떡볶이를 만드는 각각의 요리라는 프로세스에도, 밥짓기, 재료 볶기, 끓이기 등등의 작업을 스레드에 비유할 수 있습니다. 같은 작업을 좀 더 빠르게 처리하기 위해 여러 개의 스레드를 생성하기도 합니다.

프로세스와 스레드의 차이가 명확하게 이해되시나요? 아래 그림을 예로 들자면, 프로세스는 김밥, 떡볶이를 만드는 각각의 요리사와 같습니다. 이들은 각자의 전용 주방공간에서 밥짓기, 재료 볶기, 끓이기 등등의 작업를 병렬적으로 수행합니다. 도마, 불판 등 주방공간은 각각의 작업에 공유되지만, 요리사끼리 주방공간을 공유하지는 않습니다. 마찬가지로 프로세스도 자신만의 전용 메모리공간(Heap)을 가집니다. 이때 해당 프로세스 내의 스레드들은 이 메모리공간을 공유합니다. 그러나 다른 프로세스와 공유하지는 않습니다.

스레드의 사용은 프로그램마다 다르지만, 가벼운 프로그램은 하나의 스레드를 가지기도 합니다.

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/Untitled_4_XndfEVu.max-800x600.png)

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/Untitled_5_WTQo1gb.max-800x600.png)


### 프로파일링(Profiling)

코드에서 시스템의 어느 부분이 느린지 혹은 어디서 RAM을 많이 사용하고 있는지를 확인하고 싶을 때 사용하는 기법인데요. 맥(mac)의 활성 상태 보기(activity monitor)나 윈도우(windows)의 작업 관리자(task manager)를 통해 현재 실행 중인 프로그램의 상태를 확인하는 작업을 코딩하는 것을 프로파일링이라고 한답니다.

파이썬에서는 다음 명령어로 확인 할 수 있습니다.


```python
%timeit
```

```python
import time
time.time()
```

```python
import timeit
timer.Timer()
```


사실 프로파일링이라고 하기에 너무 민망한 정도긴 하지만, 한번 수행해 수행 시간을 계산해봅시다.


예제를 한번 살펴 볼까요? 100까지 숫자를 배열로 만드는 방법을 각각 다른 방법으로 구현해 보고 실행 시간을 측정해 보겠습니다.







In [2]:
import timeit
        
def f1():
    s = set(range(100))

    
def f2():
    l = list(range(100))

    
def f3():
    t = tuple(range(100))


def f4():
    s = str(range(100))

    
def f5():
    s = set()
    for i in range(100):
        s.add(i)

def f6():
    l = []
    for i in range(100):
        l.append(i)
    
def f7():
    s_comp = {i for i in range(100)}

    
def f8():
    l_comp = [i for i in range(100)]
    

if __name__ == "__main__":
    t1 = timeit.Timer("f1()", "from __main__ import f1")
    t2 = timeit.Timer("f2()", "from __main__ import f2")
    t3 = timeit.Timer("f3()", "from __main__ import f3")
    t4 = timeit.Timer("f4()", "from __main__ import f4")
    t5 = timeit.Timer("f5()", "from __main__ import f5")
    t6 = timeit.Timer("f6()", "from __main__ import f6")
    t7 = timeit.Timer("f7()", "from __main__ import f7")
    t8 = timeit.Timer("f8()", "from __main__ import f8")
    print("set               :", t1.timeit(), '[ms]')
    print("list              :", t2.timeit(), '[ms]')
    print("tuple             :", t3.timeit(), '[ms]')
    print("string            :", t4.timeit(), '[ms]')
    print("set_add           :", t5.timeit(), '[ms]')
    print("list_append       :", t6.timeit(), '[ms]')
    print("set_comprehension :", t5.timeit(), '[ms]')
    print("list_comprehension:", t6.timeit(), '[ms]')

set               : 1.3126789489906514 [ms]
list              : 0.5536312410113169 [ms]
tuple             : 0.661989669999457 [ms]
string            : 0.37623183500545565 [ms]
set_add           : 4.239037232007831 [ms]
list_append       : 3.7564659009949537 [ms]
set_comprehension : 4.163061993996962 [ms]
list_comprehension: 3.8269548259995645 [ms]


이런 식으로 함수의 성능을 측정할 수 있습니다.

> 좀 더 엄밀히 말하면 **프로파일링**은 애플리케이션에서 가장 자원이 집중되는 지점을 정밀하게 찾아내는 기법입니다. **프로파일러**는 애플리케이션을 실행시키고 각각의 함수 실행에 드는 시간을 찾아내는 프로그램이에요. 즉, **코드의 병목(bottleneck)**을 찾아내고 성능을 측정해 주는 도구입니다.

`profile` 모듈, `cProfile` 모듈과 `line_profiler` 패키지를 이용하면 높은 수준의 프로파일링이 가능한 점, 참고해주세요

## 멀티태스킹 (3) Scale Up vs Scale Out

프로그램과 성능에 대한 전반적인 개념에 대해 살펴 보았습니다. 더 포괄적인 용어로 말하면 우리는 컴퓨터 자원을 활용하기 위해 자원을 Up(업그레이드, 최적화) 시킬수도 있고 자원을 Out(확장)시킬 수도 있습니다. Scale-Up은 한 대의 컴퓨터의 성능을 최적화 시키는 방법이고 Scale-Out은 여러 대의 컴퓨터를 한 대처럼 사용하는 것입니다.

이것과 관련해서는 이전에 블로그에 포스팅을 한 적이 있습니다. 이전 포스팅을 참고해주세요!

- [컴퓨터의 성능을 높이는 방법 - 스케일업(scale up)과 스케일 아웃(scale out)](https://butter-shower.tistory.com/109)


## 파이썬에서 멀티스레드 사용하기 (1) 스레드 생성

파이썬에서 멀티스레드의 구현은 `threading` 모듈을 이용합니다.

### 기본 코드

아래 코드는 음식 배달과 그릇찾기 2가지 작업을 수행하는 코드입니다.

In [3]:
class Delivery:
    def run(self):
        print("delivery")

class RetriveDish:
    def run(self):
        print("Retriving Dish")

work1 = Delivery()
work2 = RetriveDish()

def main():
    work1.run()
    work2.run()

if __name__ == '__main__':
    main()

delivery
Retriving Dish


이 일반적인 방법은 순차적으로 수행이 됩니다.

### 멀티스레드

그럼 다음과 같이 코드를 고쳐봅시다.

- threading 모듈을 Import
- 클래스에 Thread를 상속

In [4]:
from threading import *

class Delivery(Thread):
    def run(self):
        print("delivery")

class RetriveDish(Thread):
    def run(self):
        print("Retriving Dish")

work1 = Delivery()
work2 = RetriveDish()

def main():
    work1.run()
    work2.run()

if __name__ == '__main__':
    main()

delivery
Retriving Dish


아직은 잘 모르겠군요! 스레드의 생성을 확인해봅시다.

### 스레드 생성 확인

함수 이름을 출력하면 함수 객체를 확인할 수 있습니다.

In [5]:
from threading import *

class Delivery:
    def run(self):
        print("delivering")

work1 = Delivery()
print(work1.run)

class Delivery(Thread):
    def run(self):
        print("delivering")

work2 = Delivery()
print(work2.run)

<bound method Delivery.run of <__main__.Delivery object at 0x7f08401146d0>>
<bound method Delivery.run of <Delivery(Thread-6, initial)>>


위 결과값을 확인해봅시다. `work2`는 다른 스레드에 할당되었습니다. 

## 파이썬에서 멀티스레드 사용하기 (2) 스레드 생성 및 사용

### 스레드 생성

`thread` 모듈의 `Thread` 클래스를 상속받아서 구현할수도 있지만 그대로 인스턴스화하여 스레드를 생성할수도 있습니다.

인스턴스화 하려면 `Thread` 클래스에 인자로 `target`과 `args` 값을 넣어줍니다. `args`에 넣어준 파라미터는 스레드 함수의 인자로 넘어갑니다.

```python
t = Thread(target=함수이름, args=())
```

Thread 클래스에는 `start()`, `join()`과 같은 스레드 동작과 관련된 메소드가 있습니다. Thread로 실행할 함수를 정의한 후 `start()`를 통해 스레드를 실행합니다.

In [6]:
from threading import *
from time import sleep

Stopped = False

def worker(work, sleep_sec):    
    while not Stopped:   # 그만 하라고 할때까지
        print('do ', work)    # 시키는 일을 하고
        sleep(sleep_sec)    # 잠깐 쉽니다.
    print('retired..')           # 언젠가 이 굴레를 벗어나면, 은퇴할 때가 오겠지요?
        
t = Thread(target=worker, args=('Overwork', 3))    # 일꾼 스레드를 하나 생성합니다. 열심히 일하고 3초간 쉽니다.
t.start() 

do  Overwork
do  Overwork
do  Overwork
do  Overwork
do  Overwork
do  Overwork
do  Overwork
do  Overwork
do  Overwork
do  Overwork
do  Overwork
do  Overwork
do  Overwork
do  Overwork


그런데, 이 일꾼의 일을 어떻게 멈춰주면 좋을까요? t.terminate() 같은 메소드가 있느냐 하면, 없습니다.
그래서 스레드 함수가 루프를 돌때는 꼭 멈춰야 할지를 체크하는 flag(여기서는 `Stopped`)를 체크하도록 설계해야 합니다.

In [7]:
# 이 코드 블럭을 실행하기 전까지는 일꾼 스레드는 종료하지 않습니다. 
Stopped = True    # 일꾼 일 그만하라고 세팅해 줍시다. 
t.join()                    # 일꾼 스레드가 종료할때까지 기다립니다. 
print('worker is gone.')

retired..
worker is gone.


## 파이썬에서 멀티프로세스 사용하기

파이썬에서 멀티프로세스의 구현은 `multiprocessing` 모듈을 이용해서 할 수 있습니다.

### 프로세스 생성

프로세스도 스레드와 유사합니다.

`Process` 인스턴스를 만든 뒤, `target`과 `args` 파라미터에 각각 함수 이름과 함수 인자를 전달합니다.

In [8]:
import multiprocessing as mp

def delivery():
    print('delivering...')

p = mp.Process(target=delivery, args=())
p.start()

delivering...


### 프로세스 사용

Process 클래스는 `start()`, `join()`, `terminate()` 같은 프로세스 동작 관련 메소드가 있습니다.

```python
p = mp.Process(target=delivery, args=())
p.start() # 프로세스 시작
p.join() # 실제 종료까지 기다림 (필요시에만 사용)
p.terminate() # 프로세스 종료
```

## 파이썬에서 스레드/프로세스 풀 사용하기

지금까지 스레드와 프로세스를 생성하는 방법을 배웠습니다. 사실 멀티스레드/프로세스 작업을 할 때 가장 많은 연산이 필요한 작업은 바로 이런 스레드나 프로세스를 생성하고 종료하는 일입니다. 특히 스레드/프로세스를 사용한 뒤에는 제대로 종료해 주어야 컴퓨팅 리소스가 낭비되지 않습니다.

또 지금까지 본 것처럼 하나씩 하나씩 실행한다고 전체적인 프로그램의 성능이 좋아지지는 않아요. 오히려 더 번거로울 수 있습니다. 그래서 실제로 사용할 때에는 스레드/프로세스 풀을 사용해서 생성합니다.

"풀"이란 영어로 Pool인데요, 스레드나 프로세스들로 가득찬 풀장이라고 생각하면 됩니다. 스레드 풀을 만들면 각각의 태스크들에 대해 자동으로 스레드들을 할당하고 종료합니다.

풀을 만드는 방법은 크게 2가지가 있습니다.

- Queue라는 자료구조를 사용해서 스스로 만드는 방법
- `concurrent.futures` 라이브러리의 `ThreadPoolExecutor` , `ProcessPoolExecutor` 클래스를 이용하는 방법

2가지 방법 모두 많이 사용하는 방법입니다. 아직 자료구조에 대해 배우지 않았으니 여기서는 `concurrent.futures` 모듈을 사용하는 방법만을 이용해 구현해 보겠습니다.

### `concurrent.future` 모듈 소개

이 모듈은 파이썬 3.2부터 추가된 모듈입니다. Java 언어를 배운 적이 있다면 `ThreadPoolExecutor` 에 대해 들어본 적이 있을 거예요. 이를 파이썬에서 구현한 형태라고 생각하시면 편합니다. 한국말로 "동시성 퓨처"라고 번역해서 부르기도 하는데, 기능은 크게 4가지가 있습니다.

- Executor 객체
- ThreadPoolExecutor 객체
- ProcessPoolExecutor 객체
- Future 객체

여기서는 Thread/Process Pool 구현에 필요한 부분만 살펴 보겠습니다.

#### `ThreadPoolExecutor`

`Executor` 객체를 이용하면 스레드 생성, 시작, 조인 같은 작업을 할 때, `with` 컨텍스트 관리자와 같은 방법으로 가독성 높은 코드를 구현할 수 있습니다. (프로세스 구현 방법 역시 동일하므로 설명은 스레드로만 하겠습니다.)

```python
with ThreadPoolExecutor() as executor:
    future = executor.submit(함수이름, 인자)
```

앞서 한 `Delivery` 클래스를 예시로 들면 다음과 같습니다.



In [9]:
from concurrent.futures import ThreadPoolExecutor

class Delivery:
    def run(self):
        print("delivering")
w = Delivery()

with ThreadPoolExecutor() as executor:
    future = executor.submit(w.run)

delivering


#### `multiprocessing.Pool`

multiprocessing.Pool.map을 통해 여러개의 프로세스에 특정 함수를 매핑해서 병렬처리하도록 구현하는 방법이 널리 사용됩니다.


In [10]:
from multiprocessing import Pool
from os import getpid

def double(i):
    print("I'm process ", getpid())    # pool 안에서 이 메소드가 실행될 때 pid를 확인해 봅시다.
    return i * 2

with Pool() as pool:
      result = pool.map(double, [1, 2, 3, 4, 5])
      print(result)

I'm process I'm process I'm process I'm process I'm process     24871248732487424875 



24872
[2, 4, 6, 8, 10]



위에서 `double(i)`이라는 메소드가 pool을 통해 각각 다른 pid를 가진 프로세스들 위에서 multiprocess로 실행되었다는 것을 확인할 수 있습니다.

프로그래밍이 처음이신 분에게는 조금 낯선 개념일 수도 있습니다. 다음 스텝에서 실제 프로젝트로 구현해볼 예정이니 그때 조금 더 이해할 수 있을 거예요.

## 실전 예제 (1) futures 모듈

이번 시간에는 concurrent.futures 모듈의 ProcessPoolExecutor 를 이용해 병렬 프로그래밍을 연습해 보도록 하겠습니다.

시작하기 전에, 아래 링크를 들어가서 읽어보세요.

- [concurrent.futures - Launching parallel tasks - Python 3.8.3 documentation](https://docs.python.org/ko/3/library/concurrent.futures.html)

## 실전 예제 (2) 튜토리얼

우리카 다룰 부분은 크게 두가지입니다.

- `Executor` 객체의 `map()` 함수
- `ProcessPoolExecutor` 부분

우리는 `concurrent.futures` 모듈의 `ProcessPoolExecutor`를 이용해서 멀티프로세스를 구현해 연습해보겠습니다.

아래는 위 문서의 ProcessPoolExecutor 예제에 실린 코드입니다.

```python
import math
import concurrent

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

def main():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))

if __name__ == '__main__':
    main()
```

하나씩 뜯어봅시다!

### 1. 문제

소수(prime) 판별 문제로 PRIMES 변수에 선언된 숫자들이 소수인지 아닌지를 판별합니다.

In [11]:
import math
import concurrent

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

### 2. 소수 판별 함수 `is_prime`

소수를 판별하는 로직을 함수로 구현했습니다.

In [13]:
def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

### 3. 소수 판별 함수 호출

`main()` 함수를 따로 작성하여 소수 판별 함수(`is_prime`)를 호출하였습니다.

맵-리듀스(map-reduce) 스타일로 코드를 작성하고 `map()` 함수를 `ProcessPoolExecutor()` 인스턴스에서 생성된 `executor`에서 실행시킵니다.

`concurrent.futures` 라이브러리의 프로세스 풀에서 동작하게 하기 위해 `with` 문을 써서 구현했습니다.

In [14]:
def main():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))

병렬처리와 단일처리의 비교를 위해 코드를 아래와 같이 수정해줍시다.

- 프로파일링을 위한 시간 계산 코드 추가
- 단일 처리로 수행했을 때의 코드를 추가. 단일 처리 프로파일링을 위한 시간 계산 코드 추가.

In [15]:
import time

def main():
    print("병렬처리 시작")
    start = time.time()
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))
    end = time.time()
    print("병렬처리 수행 시각", end-start, 's')

    start = time.time()
    for number, prime in zip(PRIMES, map(is_prime, PRIMES)):
        print('%d is prime: %s' % (number, prime))
    end = time.time()
    print("단일처리 수행 시각", end-start, 's')

### 전체 코드 RUN.


In [16]:
main()

병렬처리 시작
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
병렬처리 수행 시각 0.39015698432922363 s
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
단일처리 수행 시각 1.8112156391143799 s


지금까지 파이썬에서 멀피프로세싱/멀티스레딩 하는 튜토리얼을 살펴보았습니다. :)



In [17]:
from IPython.core.display import display, HTML
display(HTML("<style> .container{width:90% !important;}</style>"))